In [1]:
import pandas as pd
import os
import random
import shutil
import cv2

In [2]:
videos = pd.read_csv("./Dataset/Drone/train_videos.txt", header=None)
videos

,0
0,00_01_52_to_00_01_58.mp4
1,00_02_45_to_00_03_10_cut.mpg
2,00_06_10_to_00_06_27.mp4
3,00_09_30_to_00_10_09.mp4
4,00_10_09_to_00_10_40.mp4
...,...
72,two_parrot_disco_1.avi
73,two_uavs_plus_airplane.avi
74,2019_10_16_C0003_5043_mavic.mp4
75,GOPR5842_002.mp4


In [3]:
index = [i for i in range(77)]
val = []
while len(val) < 77 * 0.2:
    x = random.choice(index)
    if x not in val:
        val.append(x)

test = []
while len(test) < 77 * 0.2:
    x = random.choice(index)
    if x not in val and x not in test:
        test.append(x)

train = [i for i in index if i not in test and i not in val]

val, test, train

([20, 50, 40, 73, 24, 9, 56, 53, 10, 18, 76, 43, 68, 0, 70, 4],
 [12, 15, 3, 19, 75, 61, 6, 30, 17, 55, 26, 1, 67, 28, 48, 74],
 [2,
  5,
  7,
  8,
  11,
  13,
  14,
  16,
  21,
  22,
  23,
  25,
  27,
  29,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  41,
  42,
  44,
  45,
  46,
  47,
  49,
  51,
  52,
  54,
  57,
  58,
  59,
  60,
  62,
  63,
  64,
  65,
  66,
  69,
  71,
  72])

In [4]:
split = [train, val, test]
split_label = ['train/', 'val/', 'test/']

In [5]:
data_path = "./Dataset/Drone/train_images/"
yolo_path = "./Dataset/Yolo/images/"
data_ann  = "./Dataset/Drone/annotations/"
yolo_ann  = "./Dataset/Yolo/labels/"

In [6]:
index = 0
for i in split:

    d = split_label[index]
    index += 1

    for j in i:
        for k in os.listdir(data_path + videos[0][j][:-4] + '/'):

            shutil.copy(data_path + videos[0][j][:-4] + '/' + k, yolo_path + d + videos[0][j][:-4] + '_' + k)

In [7]:
index = 0
for i in split:

    d = split_label[index]
    index += 1

    for j in i:

        txt_file = data_ann + videos[0][j][:-3] + 'txt'
        ref_img = cv2.imread(yolo_path + d + videos[0][j][:-4] + '_frame_0000.jpg') 
        ref_h, ref_w = ref_img.shape[0:2]

        with open(txt_file, 'r') as fi:

            for line in fi.readlines()[::5]:

                line = line.split()
                label_file = yolo_ann + d + videos[0][j][:-4] + '_frame_' + '0' * (4 - len(line[0])) + str(int(line[0])) + '.txt'

                with open(label_file, 'w') as wr:
                    for l in range(int(line[1])):

                        x, y, w, h = list(map(int, line[l*5 + 2: (l+1)*5 + 1]))
                        x_c = (x + w/2) / ref_w 
                        y_c = (y + h/2) / ref_h 
                        w /= ref_w
                        h /= ref_h
                        wr.write(f'0 {x_c} {y_c} {w} {h}')
                    wr.close()
            fi.close()

In [8]:
train_images = os.listdir(yolo_path + 'train/')
train_labels = os.listdir(yolo_ann  + 'train/')
len(train_images), len(train_labels)

(13690, 13690)

In [9]:
for i in train_labels:
    if i[:-4]+'.jpg' not in train_images:
        os.remove(yolo_ann + 'train/' + i)
        print(i, 'removed')

In [10]:
val_images = os.listdir(yolo_path + 'val/')
val_labels = os.listdir(yolo_ann  + 'val/')
len(val_images), len(val_labels)

(3578, 3579)

In [11]:
test_images = os.listdir(yolo_path + 'test/')
test_labels = os.listdir(yolo_ann  + 'test/')
len(test_images), len(test_labels)

(4070, 4073)

In [12]:
for i in test_labels:
    if i[:-4]+'.jpg' not in test_images:
        os.remove(yolo_ann + 'test/' + i)
        print(i, 'removed')

2019_10_16_C0003_3633_inspire_frame_1415.txt removed
2019_10_16_C0003_3633_inspire_frame_1420.txt removed
2019_10_16_C0003_3633_inspire_frame_1425.txt removed
